In [ ]:
%matplotlib inline

# Input Datasets  

In [ ]:
import pandas
data_folder = '../data'
date_format='%Y-%M-%d' #truncate datetimes to dates
invoices = pandas.read_csv(data_folder + '/invoice.csv', na_values='inf', 
                           parse_dates=['invoice_date', 'due_date', 'cleared_date'], date_format=date_format)
payments = pandas.read_csv(data_folder + '/invoice_payments.csv', na_values='inf',
                           parse_dates=['transaction_date'], date_format=date_format)

## Dataset Definitions & Relationships

We have two input datasets: invoices and their payments.
- Payments are amounts in time, which are directly mapped to companies. 
- Invoices can have multiple payments, but usually only have 1. 

In [ ]:
invoices.dtypes

In [ ]:
payments.dtypes

In [ ]:
#The join key will be invoice_id, so it must be unique (and it is).
invoices.id.value_counts(dropna=False).value_counts(dropna=False)\
.to_frame(name="ids").rename_axis('invoices_per_id')

In [ ]:
#all payments are represented in both datasets 
len(set(payments.invoice_id) - set(invoices.id))

In [ ]:
#7% of invoices do not have payments yet
len(set(invoices.id) - set(payments.invoice_id))/invoices.__len__()

In [ ]:
#invoices usually have one payment but may have more
payments.invoice_id.value_counts(dropna=False).value_counts(dropna=False, normalize=True)\
.to_frame(name="invoices").rename_axis('payments_per_invoice')

## Entity Definitions & Relationships

- Company: business entity for which Tesorio is forecasting cash collected. There are only two. Each company collects using multiple currencies from multiple customers. 
- Account: **In this limited dataset, accounts and companies are synonymous, so we ignore accounts.**  
- Customer: metadata about an invoice which is specific to each company. 

In [ ]:
invoices.groupby("company_id")[["customer_id","currency"]].nunique()

In [ ]:
invoices.groupby("customer_id").company_id.nunique().value_counts()\
.to_frame(name='customers').rename_axis('companies_per_customer')

In [ ]:
invoices.groupby("company_id").account_id.nunique().to_frame(name="unique_accounts")

In [ ]:
invoices.groupby("account_id").company_id.nunique().value_counts()\
.to_frame(name='count').rename_axis('companies_per_account')

## Data Cleaning Needs

### Payments

Transaction data begins in 2011 and ends on 2021-01-31. We will assume this is when the data was pulled. 

In [ ]:
payments.__len__()

In [ ]:
payment_stats = payments.describe(exclude='int')
payment_stats.loc['% populated'] = payment_stats.loc['count']/payments.__len__()
payment_stats

In [ ]:
last_transaction_date = payments.transaction_date.max()
last_transaction_date

In [ ]:
#converted_amount is reliable
(((payments.amount * payments.root_exchange_rate_value) - payments.converted_amount).abs()).max()

In [ ]:
payments.select_dtypes(include='float').hist(bins=50, figsize=(12, 3), layout=(1,3))

### Invoices

In [ ]:
#to compare to payments. Are we holding the customer accountable to USD or their own currency?
invoices['converted_amount'] = invoices.amount_inv * invoices.root_exchange_rate_value

In [ ]:
invoices_stats = invoices.describe(exclude='int')
invoices_stats.loc['% populated'] = invoices_stats.loc['count']/invoices.__len__()
invoices_stats

In [ ]:
#opened outside of payment data time period - need to filter 
(invoices.loc[invoices.invoice_date>payments.transaction_date.max()].__len__(), 
invoices.loc[invoices.invoice_date<payments.transaction_date.min()].__len__())

### Exchange Rate

Exchange rates vary for both payments and open invoices. Customers would expect to pay the amount they were originally invoiced in their own currency, not the USD amount originally invoiced. Therefore, we should use raw amounts to determine how much is paid vs due. 

In [ ]:
# USD is not is always 1 - it varies a lot
currency_ranges = invoices.groupby("currency").root_exchange_rate_value.describe(percentiles=[])
(currency_ranges['max']/currency_ranges['min']).sort_values().plot(kind='bar', title="Exchange Rate Spread Ratio")

In [ ]:
# 1.2% of USD invoices have an exchange rate unequal to 1
invoices_usd = invoices.query("currency=='USD'").copy()
invoices_usd['exchange_rate_is_1'] = invoices_usd['root_exchange_rate_value'] == 1
1 - invoices_usd.exchange_rate_is_1.mean()

In [ ]:
# USD exchange rate variations from 1 tend to be invoices which took longer to clear
# This suggests that the invoice exchange rate is "current state data." 
time_to_clear = invoices_usd.cleared_date - invoices_usd.invoice_date
invoices_usd['months_to_clear'] = time_to_clear.map(lambda t: round(t.days/30))

In [ ]:
invoices_usd.groupby("exchange_rate_is_1").months_to_clear.agg(['mean','count'])

In [ ]:
invoices_usd.groupby(invoices_usd.months_to_clear.clip(upper=13, lower=-1))\
.exchange_rate_is_1.mean().plot(title='% of USD Invoices With Exchange Rate Equal to 1', figsize=(12,3))

### Cleared vs Open 

All invoices have a date cleared. 
When an invoice is open, the date cleared is set to the future, and seems to be an assumed value. 

In [ ]:
invoices.loc[invoices.cleared_date.isnull()].__len__()

In [ ]:
invoices.status.value_counts(normalize=True, dropna=False).to_frame(name="% of Invoices")

In [ ]:
invoices.loc[invoices.cleared_date.isnull() != (invoices.status == 'OPEN'),['status','cleared_date']]\
.value_counts(dropna=False)

In [ ]:
#all open invoices have the same cleared date
invoices.loc[invoices.status == 'OPEN'].cleared_date.value_counts(dropna=False)

In [ ]:
invoices.loc[invoices.status == 'OPEN', ['invoice_date','due_date']].max()

In [ ]:
(invoices.cleared_date>last_transaction_date).mean()

### Merging & Checking for Consistency

- 18% of payments are partial. 
- No payments are more than their invoices. 
- Exchange rates vary across payments.
- Companies are consistent between payments and invoices, when payments are present. 
- Amounts are more consistent in their original currencies than in USD

In [ ]:
invoice_payments = invoices.rename(columns={"id":"invoice_id","amount_inv":"amount"})\
.merge(payments, on="invoice_id", how='left', suffixes=('_inv', '_pmt'))

In [ ]:
invoice_payments.invoice_id.nunique()

In [ ]:
duplicated_columns = [col.replace('_pmt','') for col in invoice_payments.columns if col.endswith('_pmt')]
for col in  duplicated_columns:
    inconsistent_rows = invoice_payments.loc[invoice_payments[col + '_pmt']!=invoice_payments[col + '_inv']]
    print(f"{col}: {inconsistent_rows.__len__()/invoice_payments.__len__()} inconsistent rows in merged dataset")

In [ ]:
invoice_payments.query("company_id_pmt!=company_id_inv").company_id_pmt.value_counts(dropna=False)

In [ ]:
invoice_payments.query("amount_pmt!=amount_inv")[['amount_pmt','amount_inv']].describe()

In [ ]:
#no payment is more than the invoice amount in the original currency
invoice_payments.loc[invoice_payments.amount_pmt>invoice_payments.amount_inv].__len__()

In [ ]:
#converting to USD creates payments that are higher than invoice totals
invoice_payments.loc[invoice_payments.converted_amount_pmt>invoice_payments.converted_amount_inv].__len__()

In [ ]:
invoice_payments_rollup = invoice_payments.groupby("invoice_id", 
                                    as_index=False).agg({"amount_pmt":['sum','count'],
                                                         "transaction_date":['min','max']})
invoice_payments_rollup.columns = invoice_payments_rollup.columns.to_flat_index().map('_'.join)
payment_totals = invoices.merge(invoice_payments_rollup, how="left", left_on="id", right_on="invoice_id_")
payment_totals['amount_remaining'] = payment_totals.amount_inv - payment_totals.amount_pmt_sum

#### Cleared invoices 

Invoices with cleared dates or statuses can still have amounts remaining. 1% of cleared invoices were overpaid.

In [ ]:
#For invoices with a cleared date, the totals do not match 10% of the time
cleared_invoices = payment_totals.loc[payment_totals.cleared_date.isnull()==False].copy()
1 - (cleared_invoices.amount_remaining==0).mean()

In [ ]:
#For invoices with "cleared" status, totals do not match 7% of the time
cleared_invoices = payment_totals.loc[payment_totals.status=='CLEARED'].copy()
1 - (cleared_invoices.amount_remaining==0).mean()

In [ ]:
# 1% of cleared invoices have been overpaid according to payments data. 
(cleared_invoices.amount_remaining<0).mean()

In [ ]:
#only half of these seem to be late fees
overpayments = cleared_invoices.query("amount_remaining<0")
(overpayments.cleared_date>overpayments.due_date).mean()

#### Fully collected invoices

- Can still be open (rarely). 
- Almost always cleared with the last transaction, after converting to monthly periods. 
- These may represent late fees. 

In [ ]:
collected_invoices = payment_totals.query("amount_remaining==0").copy()
collected_invoices.status.value_counts(normalize=True, dropna=False)

In [ ]:
(collected_invoices.cleared_date-collected_invoices.transaction_date_max).describe()

In [ ]:
(collected_invoices.cleared_date.dt.to_period('M')<collected_invoices.transaction_date_max.dt.to_period('M'))\
.mean()

In [ ]:
(collected_invoices.cleared_date.dt.to_period('M')>collected_invoices.transaction_date_max.dt.to_period('M'))\
.mean()

## Transforming Dates to Quantities

In [ ]:
invoice_time_allowed = invoices.due_date - invoices.invoice_date
invoice_time_open = invoices.cleared_date - invoices.invoice_date
invoice_time_late = invoice_time_open - invoice_time_allowed

In [ ]:
invoices['days_allowed'] = invoice_time_allowed.map(lambda t: t.days if not pandas.isnull(t) else None)
invoices['days_open'] = invoice_time_open.map(lambda t: t.days if not pandas.isnull(t) else None)
invoices['days_late'] = invoice_time_late.map(lambda t: t.days if not pandas.isnull(t) else None)

invoices['months_allowed'] = (invoices.due_date.dt.to_period('M') - invoices.invoice_date.dt.to_period('M'))
invoices.months_allowed = invoices.months_allowed.map(lambda m: m.n if not pandas.isnull(m) else None)
invoices['months_open'] = (invoices.cleared_date.dt.to_period('M') - invoices.invoice_date.dt.to_period('M'))
invoices.months_open = invoices.months_open.map(lambda m: m.n if not pandas.isnull(m) else None)
invoices['months_late'] = (invoices.cleared_date.dt.to_period('M') - invoices.due_date.dt.to_period('M'))
invoices.months_late = invoices.months_late.map(lambda m: m.n if not pandas.isnull(m) else None)

## Break Down Invoices By Period

Create periods from invoice date to close date
Rolling payment window: due_date - current period
Rolling days open: cleared_date - current period 

## Assumptions

- Invoices that are cleared will have their total amount paid off, no more, no less
- Invoices will only be scored if they are open at that point in time 
- 

# Metadata Calculations & Cleaning

Totals, Uniques, Averages, Ranges, Outliers, Missings
Variables: Invoices, USD Amounts, Cleared/Open, Due Date, Invoice Date, Transaction Date, Customers, Companies, Accounts

# Notes

## Notable entities

e.g. customers with notable values

## Sparsity

### Entities

### Date Periods

## Trends Over Time

# Analysis

## Business Motivation

Cash collections don't follow due dates

## Data Science Benchmark

Define & Quantify: customers' mean absolute % error each period from cash due.

## Data Science Target

Best outcome variable? 
Days late
Days open (Total and Rolling)
Days Open as a % of Payment Window (Total and Rolling)
Days Late as a % of Payment Window (Total and Rolling)